##### Copyright 2024 Google LLC.

In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Day 1 - Prompting

Welcome to the Kaggle 5-day Generative AI course!

This notebook will show you how to get started with the Gemini API and walk you through some of the example prompts and techniques that you can also read about in the Prompting whitepaper. You don't need to read the whitepaper to use this notebook, but the papers will give you some theoretical context and background to complement this interactive notebook.


## Before you begin

In this notebook, you'll start exploring prompts and prompt parameters using the Python SDK and AI Studio. For some inspiration, you might enjoy exploring some apps that have been built using the Gemini family of models. Here are a few that we like, and we think you will too.

* [TextFX](https://textfx.withgoogle.com/) is a suite of AI-powered tools for rappers, made in collaboration with Lupe Fiasco,
* [SQL Talk](https://sql-talk-r5gdynozbq-uc.a.run.app/) shows how you can talk directly to a database using the Gemini API,
* [NotebookLM](https://notebooklm.google/) uses Gemini models to build your own personal AI research assistant.


### A note on the Gemini API and Vertex AI

In the whitepapers, most of the example code uses the Enterprise [Vertex AI platform](https://cloud.google.com/vertex-ai). In contrast, this notebook, along with the others in this series, will use the [Gemini Developer API](https://ai.google.dev/gemini-api/) and [AI Studio](https://aistudio.google.com/).

Both APIs provide access to the Gemini family of models, and the code to interact with the models is very similar. Vertex provides a world-class platform for enterprises, governments and advanced users that need powerful features like data governance, ML ops and deep Google Cloud integration.

AI Studio is free to use and only requires a compatible Google account to log in and get started. It is deeply integrated with the Gemini API, which comes with a generous [free tier](https://ai.google.dev/pricing) that you can use to run the code in these exercises.

If you are already set up with Google Cloud, you can check out the [Enterprise Gemini API](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference) through Vertex AI, and run the samples directly from the supplied whitepapers.

## Get started with Kaggle notebooks

If this is your first time using a Kaggle notebook, welcome! You can read about how to use Kaggle notebooks [in the docs](https://www.kaggle.com/docs/notebooks).

First, you will need to phone verify your account at kaggle.com/settings.

![](https://storage.googleapis.com/kaggle-media/Images/5dgai_0.png)

To run this notebook, as well as the others in this course, you will need to make a copy, or fork, the notebook. Look for the `Copy and Edit` button in the top-right, and **click it** to make an editable, private copy of the notebook. It should look like this one:

![Copy and Edit button](https://storage.googleapis.com/kaggle-media/Images/5gdai_sc_1.png)

Your copy will now have a ▶️ **Run** button next to each code cell that you can press to execute that cell. These notebooks are expected to be run in order from top-to-bottom, but you are encouraged to add new cells, run your own code and explore. If you get stuck, you can try the `Factory reset` option in the `Run` menu, or head back to the original notebook and make a fresh copy.

![Run cell button](https://storage.googleapis.com/kaggle-media/Images/5gdai_sc_2.png)

### Problems?

If you have any problems, head over to the [Kaggle Discord](https://discord.com/invite/kaggle), find the [`#5dgai-q-and-a` channel](https://discord.com/channels/1101210829807956100/1303438695143178251) and ask for help.

## Get started with the Gemini API

All of the exercises in this notebook will use the [Gemini API](https://ai.google.dev/gemini-api/) by way of the [Python SDK](https://pypi.org/project/google-generativeai/). Each of these prompts can be accessed directly in [Google AI Studio](https://aistudio.google.com/) too, so if you would rather use a web interface and skip the code for this activity, look for the <img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> AI Studio link on each prompt.

Next, you will need to add your API key to your Kaggle Notebook as a Kaggle User Secret.

![](https://storage.googleapis.com/kaggle-media/Images/5dgai_1.png)
![](https://storage.googleapis.com/kaggle-media/Images/5dgai_2.png)
![](https://storage.googleapis.com/kaggle-media/Images/5dgai_3.png)
![](https://storage.googleapis.com/kaggle-media/Images/5dgai_4.png)

### Install the SDK

In [2]:
%pip install -U -q "google-generativeai>=0.8.3"

Note: you may need to restart the kernel to use updated packages.


In [3]:
import google.generativeai as genai
from IPython.display import HTML, Markdown, display

### Set up your API key

To run the following cell, your API key must be stored it in a [Kaggle secret](https://www.kaggle.com/discussions/product-feedback/114053) named `GOOGLE_API_KEY`.

If you don't already have an API key, you can grab one from [AI Studio](https://aistudio.google.com/app/apikey). You can find [detailed instructions in the docs](https://ai.google.dev/gemini-api/docs/api-key).

To make the key available through Kaggle secrets, choose `Secrets` from the `Add-ons` menu and follow the instructions to add your key or enable it for this notebook.

In [4]:
import os

def get_api_key():
    try:
        # 尝试导入 kaggle_secrets (仅在 Kaggle 环境中可用)
        from kaggle_secrets import UserSecretsClient
        return UserSecretsClient().get_secret("GOOGLE_API_KEY")
    except ImportError:
        # 如果不在 Kaggle 环境中,从环境变量获取
        api_key = os.getenv("GOOGLE_API_KEY")
        if not api_key:
            raise ValueError("请设置 GOOGLE_API_KEY 环境变量或在 Kaggle 中配置密钥")
        return api_key

GOOGLE_API_KEY = get_api_key()
genai.configure(api_key=GOOGLE_API_KEY)

In [5]:
import os


# 设置HTTP代理
proxy = os.getenv("HTTP_PROXY")
if proxy:
    print(f"当前使用的代理: {proxy}")
    os.environ['HTTP_PROXY'] = f'http:{proxy}'
    os.environ['HTTPS_PROXY'] = f'http:{proxy}'


In [6]:
for model in genai.list_models():
    print(f"模型名称: {model.name}")
    print(f"模型基本信息: {model}")
    print("-" * 50)

模型名称: models/chat-bison-001
模型基本信息: Model(name='models/chat-bison-001',
      base_model_id='',
      version='001',
      display_name='PaLM 2 Chat (Legacy)',
      description='A legacy text-only model optimized for chat conversations',
      input_token_limit=4096,
      output_token_limit=1024,
      supported_generation_methods=['generateMessage', 'countMessageTokens'],
      temperature=0.25,
      max_temperature=None,
      top_p=0.95,
      top_k=40)
--------------------------------------------------
模型名称: models/text-bison-001
模型基本信息: Model(name='models/text-bison-001',
      base_model_id='',
      version='001',
      display_name='PaLM 2 (Legacy)',
      description='A legacy model that understands text and generates text as an output',
      input_token_limit=8196,
      output_token_limit=1024,
      supported_generation_methods=['generateText', 'countTextTokens', 'createTunedTextModel'],
      temperature=0.7,
      max_temperature=None,
      top_p=0.95,
      top_k=40

If you received an error response along the lines of `No user secrets exist for kernel id ...`, then you need to add your API key via `Add-ons`, `Secrets` **and** enable it.

![Screenshot of the checkbox to enable GOOGLE_API_KEY secret](https://storage.googleapis.com/kaggle-media/Images/5gdai_sc_3.png)

### Run your first prompt

In this step, you will test that your API key is set up correctly by making a request. The `gemini-1.5-flash` model has been selected here.

In [7]:
flash = genai.GenerativeModel('gemini-1.5-flash')
response = flash.generate_content("Explain AI to me like I'm a kid.")
print(response.text)

Imagine you have a really smart friend who loves to learn new things. That friend is like AI, or Artificial Intelligence. 

AI is like a computer that can think and learn like a human. It can do lots of things, like:

* **Play games:** AI can play games like chess and even video games!
* **Answer questions:** AI can answer questions like "What's the capital of France?" or "What's the weather like today?"
* **Write stories:** AI can even write stories, poems, and songs!
* **Help you with your homework:** AI can help you find information and understand difficult topics.

AI learns by looking at lots of data, like pictures, words, and videos. The more it learns, the smarter it gets!

Think of it like learning a new language. The more you practice, the better you get! AI does the same thing with information, becoming smarter over time.

It's important to remember that AI is still learning. It might make mistakes sometimes, just like we do! But it's getting better all the time, and it's alr

The response often comes back in markdown format, which you can render directly in this notebook.

In [8]:
Markdown(response.text)

Imagine you have a really smart friend who loves to learn new things. That friend is like AI, or Artificial Intelligence. 

AI is like a computer that can think and learn like a human. It can do lots of things, like:

* **Play games:** AI can play games like chess and even video games!
* **Answer questions:** AI can answer questions like "What's the capital of France?" or "What's the weather like today?"
* **Write stories:** AI can even write stories, poems, and songs!
* **Help you with your homework:** AI can help you find information and understand difficult topics.

AI learns by looking at lots of data, like pictures, words, and videos. The more it learns, the smarter it gets!

Think of it like learning a new language. The more you practice, the better you get! AI does the same thing with information, becoming smarter over time.

It's important to remember that AI is still learning. It might make mistakes sometimes, just like we do! But it's getting better all the time, and it's already helping us in many ways! 


### Start a chat

The previous example uses a single-turn, text-in/text-out structure, but you can also set up a multi-turn chat structure too.

In [9]:
chat = flash.start_chat(history=[])
response = chat.send_message('Hello! My name is Zlork.')
Markdown(response.text)

Hello Zlork! It's nice to meet you.  What can I do for you today? 😊 


In [10]:
response = chat.send_message('Can you tell something interesting about dinosaurs?')
Markdown(response.text)

Okay, Zlork, here's something interesting about dinosaurs:

**Did you know that some dinosaurs had feathers?**

While we often picture dinosaurs as scaly beasts, many species, especially those closely related to birds, sported feathers!  These feathers weren't always for flight, but might have been used for insulation, display, or even brooding their eggs. 

There's evidence for feathered dinosaurs in fossils like *Archaeopteryx* and *Microraptor*, and even some of the large, meat-eating theropods like *Tyrannosaurus Rex* might have had feathers on their bodies. 

Isn't that fascinating? It really changes our understanding of these ancient giants! 


In [11]:
# While you have the `chat` object around, the conversation state
# persists. Confirm that by asking if it knows my name.
response = chat.send_message('Do you remember what my name is?')
Markdown(response.text)

Of course, Zlork! I remember your name. 😊  It's always nice to be able to address someone by their name. 


### Choose a model

The Gemini API provides access to a number of models from the Gemini model family. Read about the available models and their capabilities on the [model overview page](https://ai.google.dev/gemini-api/docs/models/gemini).

In this step you'll use the API to list all of the available models.

In [12]:
for model in genai.list_models():
  print(model.name)

models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/embedding-001
models/text-embedding-004
models/aqa


The [`models.list`](https://ai.google.dev/api/models#method:-models.list) response also returns additional information about the model's capabilities, like the token limits and supported parameters.

In [13]:
for model in genai.list_models():
  if model.name == 'models/gemini-1.5-flash':
    print(model)
    break

Model(name='models/gemini-1.5-flash',
      base_model_id='',
      version='001',
      display_name='Gemini 1.5 Flash',
      description='Fast and versatile multimodal model for scaling across diverse tasks',
      input_token_limit=1000000,
      output_token_limit=8192,
      supported_generation_methods=['generateContent', 'countTokens'],
      temperature=1.0,
      max_temperature=2.0,
      top_p=0.95,
      top_k=40)


## Explore generation parameters



### Output length

When generating text with an LLM, the output length affects cost and performance. Generating more tokens increases computation, leading to higher energy consumption, latency, and cost.

To stop the model from generating tokens past a limit, you can specify the `max_output_length` parameter when using the Gemini API. Specifying this parameter does not influence the generation of the output tokens, so the output will not become more stylistically or textually succinct, but it will stop generating tokens once the specified length is reached. Prompt engineering may be required to generate a more complete output for your given limit.

In [14]:
short_model = genai.GenerativeModel(
    'gemini-1.5-flash',
    generation_config=genai.GenerationConfig(max_output_tokens=200))

response = short_model.generate_content('写一篇1000字的文章，论述橄榄在现代社会中的重要性。')

In [15]:
Markdown(response.text)

## 橄榄：一种古老的果实，现代生活的宝藏

橄榄，一种看似平凡的果实，却拥有着悠久的历史和丰富的文化内涵，在现代社会中依然发挥着不可替代的作用。从古代文明的象征到现代生活的健康之选，橄榄以其独特的味道、营养价值和文化意义，深深地融入人类的社会生活，展现着一种古老的智慧和现代的魅力。

**1. 橄榄：历史的回声**

橄榄的起源可以追溯到数千年前，它在古代地中海地区被广泛种植和食用。在古希腊和罗马文化中，橄榄树被视为神圣的树木，象征着和平、智慧和胜利。橄榄枝更是和平的象征，被用作奖励运动员、庆祝胜利和缔结和平条约的象征。

在古代世界中，橄榄不仅是食物，更是重要的经济作物

In [16]:
response = short_model.generate_content('写一首短诗，论述橄榄在现代社会中的重要性。')
Markdown(response.text)

橄榄树，古老而坚韧，
见证了时代的变迁。
如今它被用来制作食品，
一种美味的健康之选。

从古老的药草到现代的食谱，
橄榄树持续为人类服务。
它带来了和平、健康和繁荣，
一个真实的象征，世代相传。

所以让我们珍惜橄榄树，
它提供的珍贵礼物，
一种来自自然的美味佳肴，
世代相传。

Explore with your own prompts. Try a prompt with a restrictive output limit and then adjust the prompt to work within that limit.

### Temperature

Temperature controls the degree of randomness in token selection. Higher temperatures result in a higher number of candidate tokens from which the next output token is selected, and can produce more diverse results, while lower temperatures have the opposite effect, such that a temperature of 0 results in greedy decoding, selecting the most probable token at each step.

Temperature doesn't provide any guarantees of randomness, but it can be used to "nudge" the output somewhat.

In [17]:
high_temp_model = genai.GenerativeModel(
    'gemini-1.5-flash',
    generation_config=genai.GenerationConfig(temperature=2.0))

In [18]:
for _ in range(5):
  response = high_temp_model.generate_content('尽可能地随机生成一个关于颜色的词语, 返回结果为一个词, 比如: 红色')
  if response.parts:
    print(response.text, '-' * 25)

紫色 
 -------------------------
紫罗兰色 
 -------------------------
**蓝** 
 -------------------------
紫罗兰色 
 -------------------------
天蓝色 
 -------------------------


Now try the same prompt with temperature set to zero. Note that the output is not completely deterministic, as other parameters affect token selection, but the results will tend to be more stable.

In [19]:
low_temp_model = genai.GenerativeModel(
    'gemini-1.5-flash',
    generation_config=genai.GenerationConfig(temperature=0.0))

for _ in range(5):
  response = low_temp_model.generate_content('尽可能地随机生成一个关于颜色的词语, 返回结果为一个词, 比如: 红色')
  if response.parts:
    print(response.text, '-' * 25)

青色 
 -------------------------
青色 
 -------------------------
青色 
 -------------------------
青色 
 -------------------------
青色 
 -------------------------


### Top-K 和 top-P
与温度一样，top-K 和 top-P 参数也用于控制模型输出的多样性。

Top-K 是一个正整数，它定义了从中选择输出标记的最可能标记的数量。top-K 为 1 表示选择一个标记，执行贪婪解码。

Top-P 定义概率阈值，一旦累计超过该阈值，标记将不再被选为候选标记。top-P 为 0 通常相当于贪婪解码，top-P 为 1 表示选择模型词汇表中的每个标记。

当两者均提供时，Gemini API 将首先过滤 top-K 标记，然后过滤 top-P，最后使用提供的温度从候选标记中采样。

多次运行此示例，更改设置并观察输出的变化。

In [20]:
model = genai.GenerativeModel(
    'gemini-1.5-flash-001',
    generation_config=genai.GenerationConfig(
        # These are the default values for gemini-1.5-flash-001.
        temperature=1.0,
        top_k=64,
        top_p=0.5,
    ))

story_prompt = "你是一位创意作家。写一篇关于一只猫去冒险的短篇故事。"
response = model.generate_content(story_prompt)
Markdown(response.text)

在繁华的城市街道上，在一个阳光明媚的下午，一只名叫米顿的猫正在经历着一次非凡的冒险。米顿不是一只普通的猫；他拥有着一颗好奇的心和一颗渴望探险的灵魂。

当米顿的主人，一位名叫艾丽丝的年轻女子，忙于工作时，米顿决定独自踏上一次冒险之旅。他从窗户偷偷溜了出去，踏上了未知的道路。

米顿沿着狭窄的小巷和繁忙的街道漫步，他那敏锐的感官捕捉到了周围世界的所有声音和气味。他看到了色彩鲜艳的商店橱窗，听到了汽车的鸣笛声，闻到了烤肉的香味。

当米顿继续前进时，他发现自己身处一座古老的公园里。高耸的树木投下长长的阴影，鲜艳的花朵在微风中摇曳。米顿被这宁静的美丽所吸引，他开始在草地上漫步，追逐着飞舞的蝴蝶。

当米顿在公园里玩耍时，他听到了一阵奇怪的声音。他好奇地走到声音的来源，发现一只小兔子被困在一个洞里。兔子惊恐地尖叫着，它的小腿被一根粗大的树根卡住了。

米顿的心中充满了同情，他决定帮助这只可怜的小兔子。他用自己的爪子轻轻地推着树根，直到它松动。兔子感激地从洞里爬了出来，然后跳到米顿的身边，用鼻子蹭了蹭他的脸颊。

米顿和兔子成了朋友，他们一起在公园里玩耍，直到太阳开始下山。当夜幕降临的时候，米顿意识到自己迷路了，而且他很怀念艾丽丝。

他开始寻找回家的路，但街道看起来都一样。米顿感到沮丧和迷茫，他不知道该怎么办。

就在这时，他听到了一阵熟悉的声音。那是艾丽丝在叫他的名字。米顿循声望去，看到艾丽丝站在街角，焦急地寻找着他。

米顿高兴地跑向艾丽丝，她紧紧地抱住了他，脸上充满了喜悦。米顿很高兴能回家，但他永远不会忘记他那次非凡的冒险。

从那以后，米顿继续在城市里冒险，但他总是记得要小心，而且他总是带着他新朋友兔子留下的美好回忆。米顿知道，即使在最熟悉的环境中，也总会有新的东西可以发现，新的朋友可以结交，新的冒险可以体验。

## Prompting

This section contains some prompts from the chapter for you to try out directly in the API. Try changing the text here to see how each prompt performs with different instructions, more examples, or any other changes you can think of.

### Zero-shot

Zero-shot prompts are prompts that describe the request for the model directly.

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/1gzKKgDHwkAvexG5Up0LMtl1-6jKMKe4g"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> Open in AI Studio</a>
  </td>
</table>

In [21]:
model = genai.GenerativeModel(
    'gemini-1.5-flash-001',
    generation_config=genai.GenerationConfig(
        temperature=0.1,
        top_p=1,
        max_output_tokens=5,
    ))

zero_shot_prompt = """将电影评论分为正面、中性或负面。
评论：“她”是一项令人不安的研究，揭示了如果人工智能不受控制地不断发展，人类将走向何方。我希望有更多像这部杰作这样的电影。
情绪："""

response = model.generate_content(zero_shot_prompt)

In [22]:
Markdown(response.text)

评论的情绪是 **正面

#### Enum mode

The models are trained to generate text, and can sometimes produce more text than you may wish for. In the preceding example, the model will output the label, sometimes it can include a preceding "Sentiment" label, and without an output token limit, it may also add explanatory text afterwards.

The Gemini API has an [Enum mode](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Enum.ipynb) feature that allows you to constrain the output to a fixed set of values.

这些模型经过训练可以生成文本，有时会生成比您希望的更多的文本。在前面的示例中，模型将输出标签，有时它可能包含前面的“情绪”标签，并且没有输出标记限制，它还可能在之后添加解释性文本。

Gemini API 具有枚举模式功能，可让您将输出限制为一组固定的值。

In [23]:
import enum

class Sentiment(enum.Enum):
    POSITIVE = "正面"
    NEUTRAL = "中立"
    NEGATIVE = "负面"


model = genai.GenerativeModel(
    'gemini-1.5-flash-001',
    generation_config=genai.GenerationConfig(
        response_mime_type="text/x.enum",
        response_schema=Sentiment
    ))

response = model.generate_content(zero_shot_prompt)
print(response.text)

正面


### One-shot and few-shot

Providing an example of the expected response is known as a "one-shot" prompt. When you provide multiple examples, it is a "few-shot" prompt.

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/1jjWkjUSoMXmLvMJ7IzADr_GxHPJVV2bg"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> Open in AI Studio</a>
  </td>
</table>


In [24]:
model = genai.GenerativeModel(
    'gemini-1.5-flash-latest',
    generation_config=genai.GenerationConfig(
        temperature=0.1,
        top_p=1,
        max_output_tokens=250,
    ))

few_shot_prompt = """Parse a customer's pizza order into valid JSON:

EXAMPLE:
I want a small pizza with cheese, tomato sauce, and pepperoni.
JSON Response:
```
{
"size": "small",
"type": "normal",
"ingredients": ["cheese", "tomato sauce", "peperoni"]
}
```

EXAMPLE:
Can I get a large pizza with tomato sauce, basil and mozzarella
JSON Response:
```
{
"size": "large",
"type": "normal",
"ingredients": ["tomato sauce", "basil", "mozzarella"]
}

ORDER:
"""

customer_order = "Give me a large with cheese & pineapple"


response = model.generate_content([few_shot_prompt, customer_order])
print(response.text)

```json
{
"size": "large",
"type": "normal",
"ingredients": ["cheese", "pineapple"]
}
``` 



#### JSON mode

To provide control over the schema, and to ensure that you only receive JSON (with no other text or markdown), you can use the Gemini API's [JSON mode](https://github.com/google-gemini/cookbook/blob/main/quickstarts/JSON_mode.ipynb). This forces the model to constrain decoding, such that token selection is guided by the supplied schema.

为了控制架构并确保只接收 JSON（不包含其他文本或 markdown），您可以使用 Gemini API 的 JSON 模式。这会强制模型限制解码，以便根据提供的架构选择标记。

In [25]:
import typing_extensions as typing

class PizzaOrder(typing.TypedDict):
    size: str
    ingredients: list[str]
    type: str


model = genai.GenerativeModel(
    'gemini-1.5-flash-latest',
    generation_config=genai.GenerationConfig(
        temperature=0.1,
        response_mime_type="application/json",
        response_schema=PizzaOrder,
    ))

response = model.generate_content("Can I have a large dessert pizza with apple and chocolate")
print(response.text)

{"ingredients": ["apple", "chocolate"], "size": "large", "type": "dessert"}



### Chain of Thought (CoT)

Direct prompting on LLMs can return answers quickly and (in terms of output token usage) efficiently, but they can be prone to hallucination. The answer may "look" correct (in terms of language and syntax) but is incorrect in terms of factuality and reasoning.

Chain-of-Thought prompting is a technique where you instruct the model to output intermediate reasoning steps, and it typically gets better results, especially when combined with few-shot examples. It is worth noting that this technique doesn't completely eliminate hallucinations, and that it tends to cost more to run, due to the increased token count.

As models like the Gemini family are trained to be "chatty" and provide reasoning steps, you can ask the model to be more direct in the prompt.

### 思路链 (CoT)
LLM 上的直接提示可以快速高效地（就输出 token 使用情况而言）返回答案，但容易产生幻觉。答案可能“看起来”正确（就语言和语法而言），但在事实性和推理方面却不正确。

思路链提示是一种指示模型输出中间推理步骤的技术，通常会获得更好的结果，尤其是与少量示例结合使用时。值得注意的是，这种技术并不能完全消除幻觉，而且由于 token 数量增加，运行成本往往更高。

由于像 Gemini 系列这样的模型经过训练，变得“健谈”并提供推理步骤，因此您可以要求模型在提示中更直接。

In [26]:
prompt = """当我4岁时，我的伴侣是我的3倍。现在，我20岁了。我的伴侣几岁？立即回答。"""

model = genai.GenerativeModel('gemini-1.5-flash-latest')
response = model.generate_content(prompt)

print(response.text)

当您4岁时，您的伴侣是您的3倍，意味着您的伴侣是 4 * 3 = 12 岁。

这意味着您的伴侣比您大 12 - 4 = 8 岁。

因此，现在您 20 岁，您的伴侣是 20 + 8 = **28 岁**。 



Now try the same approach, but indicate to the model that it should "think step by step".

In [27]:
prompt = """我4岁的时候，我的伴侣年龄是我的3倍。现在，我20岁了。我的伴侣几岁？我们一步步想想。"""

response = model.generate_content(prompt)
print(response.text)

让我们一步步想：

* 当你 4 岁的时候，你的伴侣是你的三倍，也就是 4 * 3 = 12 岁。
* 这意味着你的伴侣比你大 12 - 4 = 8 岁。
* 由于年龄差是固定的，所以当你 20 岁的时候，你的伴侣比你大 8 岁，也就是 20 + 8 = **28 岁**。



### ReAct: Reason and act

In this example you will run a ReAct prompt directly in the Gemini API and perform the searching steps yourself. As this prompt follows a well-defined structure, there are frameworks available that wrap the prompt into easier-to-use APIs that make tool calls automatically, such as the LangChain example from the chapter.

To try this out with the Wikipedia search engine, check out the [Searching Wikipedia with ReAct](https://github.com/google-gemini/cookbook/blob/main/examples/Search_Wikipedia_using_ReAct.ipynb) cookbook example.


> Note: The prompt and in-context examples used here are from [https://github.com/ysymyth/ReAct](https://github.com/ysymyth/ReAct) which is published under a [MIT license](https://opensource.org/licenses/MIT), Copyright (c) 2023 Shunyu Yao.


<a target="_blank" href="https://aistudio.google.com/prompts/18oo63Lwosd-bQ6Ay51uGogB3Wk3H8XMO"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> Open in AI Studio</a>


### ReAct：推理和行动
在此示例中，您将直接在 Gemini API 中运行 ReAct 提示并自行执行搜索步骤。由于此提示遵循定义明确的结构，因此有可用的框架将提示包装到更易于使用的 API 中，这些 API 会自动进行工具调用，例如本章中的 LangChain 示例。

要使用维基百科搜索引擎尝试此操作，请查看使用 ReAct 搜索维基百科食谱示例。


In [28]:
# model_instructions = """
# 通过交替思考、行动和观察步骤来解决问答任务。思考可以推断当前情况，
# 观察是从行动的输出中理解相关信息，而行动可以是以下三种类型之一：
# (1) <search>entity</search>，它在维基百科上搜索精确实体，如果存在则返回第一段。如果不存在，它将返回一些类似的实体进行搜索，您可以尝试从这些主题中搜索信息。
# (2) <lookup>keyword</lookup>，它返回当前上下文中包含关键字的下一个句子。这只会进行精确匹配，因此请保持搜索简短。
# (3) <finish>answer</finish>，它返回答案并完成任务。"""
model_instructions = """
Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation,
Observation is understanding relevant information from an Action's output and Action can be one of three types:
 (1) <search>entity</search>, which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it
     will return some similar entities to search and you can try to search the information from those topics.
 (2) <lookup>keyword</lookup>, which returns the next sentence containing keyword in the current context. This only does exact matches,
     so keep your searches short.
 (3) <finish>answer</finish>, which returns the answer and finishes the task.
"""

example1 = """Question
Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?

Thought 1
The question simplifies to "The Simpsons" character Milhouse is named after who. I only need to search Milhouse and find who it is named after.

Action 1
<search>Milhouse</search>

Observation 1
Milhouse Mussolini Van Houten is a recurring character in the Fox animated television series The Simpsons voiced by Pamela Hayden and created by Matt Groening.

Thought 2
The paragraph does not tell who Milhouse is named after, maybe I can look up "named after".

Action 2
<lookup>named after</lookup>

Observation 2
Milhouse was named after U.S. president Richard Nixon, whose middle name was Milhous.

Thought 3
Milhouse was named after U.S. president Richard Nixon, so the answer is Richard Nixon.

Action 3
<finish>Richard Nixon</finish>
"""

example2 = """Question
What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?

Thought 1
I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.

Action 1
<search>Colorado orogeny</search>

Observation 1
The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.

Thought 2
It does not mention the eastern sector. So I need to look up eastern sector.

Action 2
<lookup>eastern sector</lookup>

Observation 2
The eastern sector extends into the High Plains and is called the Central Plains orogeny.

Thought 3
The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.

Action 3
<search>High Plains</search>

Observation 3
High Plains refers to one of two distinct land regions

Thought 4
I need to instead search High Plains (United States).

Action 4
<search>High Plains (United States)</search>

Observation 4
The High Plains are a subregion of the Great Plains. From east to west, the High Plains rise in elevation from around 1,800 to 7,000 ft (550 to 2,130m).

Thought 5
High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer is 1,800 to 7,000 ft.

Action 5
<finish>1,800 to 7,000 ft</finish>
"""

case1 = """问题
音乐家兼讽刺作家 Allie Goertz 写了一首关于"辛普森一家"角色 Milhouse 的歌,Matt Groening 是以谁来给这个角色命名的?

思考 1
问题简化为"辛普森一家"角色 Milhouse 是以谁命名的。我只需要搜索 Milhouse 并找出它是以谁命名的。

行动 1
<search>Milhouse</search>

观察 1
Milhouse Mussolini Van Houten 是福克斯动画电视剧《辛普森一家》中由 Pamela Hayden 配音、Matt Groening 创作的一个常见角色。

思考 2
这段话没有说 Milhouse 是以谁命名的,也许我可以查找"named after"。

行动 2
<lookup>named after</lookup>

观察 2
Milhouse 是以美国总统理查德·尼克松命名的,尼克松的中间名是 Milhous。

思考 3
Milhouse 是以美国总统理查德·尼克松命名的,所以答案是理查德·尼克松。

行动 3
<finish>理查德·尼克松</finish>
"""

case2 = """问题
科罗拉多造山运动的东部区域延伸到的区域的海拔范围是多少?

思考 1
我需要搜索科罗拉多造山运动,找到其东部区域延伸到的地方,然后找到该区域的海拔范围。

行动 1
<search>Colorado orogeny</search>

观察 1
科罗拉多造山运动是在科罗拉多及周边地区发生的一次造山运动(造山期)。

思考 2
它没有提到东部区域。所以我需要查找 eastern sector。

行动 2
<lookup>eastern sector</lookup>

观察 2
东部区域延伸到高平原,被称为中央平原造山运动。

思考 3
科罗拉多造山运动的东部区域延伸到高平原。所以我需要搜索高平原并找到其海拔范围。

行动 3
<search>High Plains</search>

观察 3
高平原指的是两个不同的陆地区域之一。

思考 4
我需要改为搜索美国高平原。

行动 4
<search>High Plains (United States)</search>

观察 4
高平原是大平原的一个亚区域。从东到西,高平原的海拔从约1,800英尺升至7,000英尺(550至2,130米)。

思考 5
高平原的海拔从1,800英尺升至7,000英尺,所以答案是1,800至7,000英尺。

行动 5
<finish>1,800至7,000英尺</finish>
"""

# # Come up with more examples yourself, or take a look through https://github.com/ysymyth/ReAct/

To capture a single step at a time, while ignoring any hallucinated Observation steps, you will use `stop_sequences` to end the generation process. The steps are `Thought`, `Action`, `Observation`, in that order.

注意，使用gemini model api实现不了这个例子，返回结果只有单步：
```
Thought 1
I need to find the Transformers paper and then find the authors and their ages. 

Action 1
<search>Transformers NLP paper</search>
```

In [30]:
question = """Question
Who was the youngest author listed on the transformers NLP paper?"""

model = genai.GenerativeModel('gemini-1.5-flash-latest')
react_chat = model.start_chat()

# You will perform the Action, so generate up to, but not including, the Observation.
config = genai.GenerationConfig(stop_sequences=["\nObservation"])

resp = react_chat.send_message(
    [model_instructions, example1, example2, question],
    generation_config=config)
print(resp.text)

Thought 1
I need to find the Transformers paper and then find the authors and their ages. 

Action 1
<search>Transformers NLP paper</search>



Now you can perform this research yourself and supply it back to the model.

In [48]:
observation = """Observation 1
[1706.03762] Attention Is All You Need
Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely.
"""
resp = react_chat.send_message(observation, generation_config=config)
print(resp.text)

Thought 2:  I need to find the youngest author from the list. 

Action 2: <lookup>youngest</lookup> 



This process repeats until the `<finish>` action is reached. You can continue running this yourself if you like, or try the [Wikipedia example](https://github.com/google-gemini/cookbook/blob/main/examples/Search_Wikipedia_using_ReAct.ipynb) to see a fully automated ReAct system at work.

## Code prompting

### Generating code

The Gemini family of models can be used to generate code, configuration and scripts. Generating code can be helpful when learning to code, learning a new language or for rapidly generating a first draft.

It's important to be aware that since LLMs can't reason, and can repeat training data, it's essential to read and test your code first, and comply with any relevant licenses.

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/1YX71JGtzDjXQkgdes8bP6i3oH5lCRKxv"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> Open in AI Studio</a>
  </td>
</table>

In [32]:
model = genai.GenerativeModel(
    'gemini-1.5-flash-latest',
    generation_config=genai.GenerationConfig(
        temperature=1,
        top_p=1,
        max_output_tokens=1024,
    ))

# Gemini 1.5 models are very chatty, so it helps to specify they stick to the code.
code_prompt = """
编写一个 Python 函数来计算某个数的阶乘。不做解释，只提供代码。
"""

response = model.generate_content(code_prompt)
Markdown(response.text)

```python
def factorial(n):
  if n == 0:
    return 1
  else:
    return n * factorial(n-1)
```

### Code execution

The Gemini API can automatically run generated code too, and will return the output.

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/11veFr_VYEwBWcLkhNLr-maCG0G8sS_7Z"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> Open in AI Studio</a>
  </td>
</table>

In [33]:
model = genai.GenerativeModel(
    'gemini-1.5-flash-latest',
    tools='code_execution')

In [35]:
code_exec_prompt = """
计算前 14 个素数的总和。
"""

response = model.generate_content(code_exec_prompt)
Markdown(response.text)

我会计算前 14 个素数的总和。

首先，我需要定义一个素数。素数是一个大于 1 的自然数，除了 1 和它本身以外没有其他因数。

接下来，我将编写一个 Python 代码来生成前 14 个素数：


``` python
def is_prime(n):
  """判断一个数是否为素数。"""
  if n <= 1:
    return False
  for i in range(2, int(n ** 0.5) + 1):
    if n % i == 0:
      return False
  return True


primes = []
num = 2
while len(primes) < 14:
  if is_prime(num):
    primes.append(num)
  num += 1

print(f'前 14 个素数是: {primes}')
print(f'前 14 个素数的总和为: {sum(primes)}')

```
```
前 14 个素数是: [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43]
前 14 个素数的总和为: 281

```
因此，前 14 个素数的总和是 281。

In [46]:

for part in response.candidates[0].content.parts:
  print(part)
  print("-----")
    
# for part in response.candidates[0].content.parts:
#     if hasattr(part, 'text'):
#         print(part.text)  # 直接打印，不需要额外的编码解码
#     elif hasattr(part, 'code'):
#         print(part.code)
#     elif hasattr(part, 'output'):
#         print(part.code)
#     else:
#         print(part)
#     print("-----")
# for part in response.candidates[0].content.parts:
#   print(part.txt)
#   print("-----")

text: "\346\210\221\344\274\232\350\256\241\347\256\227\345\211\215 14 \344\270\252\347\264\240\346\225\260\347\232\204\346\200\273\345\222\214\343\200\202\n\n\351\246\226\345\205\210\357\274\214\346\210\221\351\234\200\350\246\201\345\256\232\344\271\211\344\270\200\344\270\252\347\264\240\346\225\260\343\200\202\347\264\240\346\225\260\346\230\257\344\270\200\344\270\252\345\244\247\344\272\216 1 \347\232\204\350\207\252\347\204\266\346\225\260\357\274\214\351\231\244\344\272\206 1 \345\222\214\345\256\203\346\234\254\350\272\253\344\273\245\345\244\226\346\262\241\346\234\211\345\205\266\344\273\226\345\233\240\346\225\260\343\200\202\n\n\346\216\245\344\270\213\346\235\245\357\274\214\346\210\221\345\260\206\347\274\226\345\206\231\344\270\200\344\270\252 Python \344\273\243\347\240\201\346\235\245\347\224\237\346\210\220\345\211\215 14 \344\270\252\347\264\240\346\225\260\357\274\232\n\n"

-----
executable_code {
  language: PYTHON
  code: "\ndef is_prime(n):\n  \"\"\"\345\210\244

While this looks like a single-part response, you can inspect the response to see the each of the steps: initial text, code generation, execution results, and final text summary.

### Explaining code

The Gemini family of models can explain code to you too.

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/1N7LGzWzCYieyOf_7bAG4plrmkpDNmUyb"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> Open in AI Studio</a>
  </td>
</table>

In [48]:
file_contents = !curl https://raw.githubusercontent.com/magicmonty/bash-git-prompt/refs/heads/master/gitprompt.sh

explain_prompt = f"""
Please explain what this file does at a very high level. What is it, and why would I use it? Answer in Chinese.

```
{file_contents}
```
"""

model = genai.GenerativeModel('gemini-1.5-flash-latest')

response = model.generate_content(explain_prompt)
Markdown(response.text)

这个文件是一个名为 `bash-git-prompt` 的 Bash 脚本，它提供了一个用于 Git 仓库的自定义提示符。这个提示符会根据当前 Git 仓库的状态，例如分支、修改、冲突等，显示不同的信息。

这个脚本可以帮助你在 Bash 中使用 Git 时，更容易地了解当前仓库的状态。例如：

*  脚本会根据当前分支显示不同的颜色。
*  脚本会显示未提交的更改数量。
*  脚本会显示当前分支是否与远程分支同步。

你可以在你的 `~/.bashrc` 文件中添加以下代码来使用这个脚本：

```bash
source /path/to/bash-git-prompt.sh
```

其中 `/path/to/bash-git-prompt.sh` 是 `bash-git-prompt.sh` 文件的路径。

你还可以使用 `GIT_PROMPT_THEME` 环境变量来设置不同的主题。例如：

```bash
export GIT_PROMPT_THEME=Solarized
```

这个脚本是一个强大的工具，可以帮助你提高在 Bash 中使用 Git 的效率。


## Learn more

To learn more about prompting in depth:

* Check out the whitepaper issued with today's content,
* Try out the apps listed at the top of this notebook ([TextFX](https://textfx.withgoogle.com/), [SQL Talk](https://sql-talk-r5gdynozbq-uc.a.run.app/) and [NotebookLM](https://notebooklm.google/)),
* Read the [Introduction to Prompting](https://ai.google.dev/gemini-api/docs/prompting-intro) from the Gemini API docs,
* Explore the Gemini API's [prompt gallery](https://ai.google.dev/gemini-api/prompts) and try them out in AI Studio,
* Check out the Gemini API cookbook for [inspirational examples](https://github.com/google-gemini/cookbook/blob/main/examples/) and [educational quickstarts](https://github.com/google-gemini/cookbook/blob/main/quickstarts/).

And please share anything exciting you have tried in the Discord!